# Satellite Entry/Exit Event Detection with Ground Sensor Field of View 

The FoV of the ground sensor is represented as different shapes.

- Circular FoV – A simple circular field of view around the sensor.
- Rectangular FoV – A rectangular field of view providing directional coverage.
- Half Power Beam Width (HPBW) FoV – A field of view based on the sensor’s Half Power Beam Width, typically used for antenna beam modeling.

The sensor's attitude can be adjusted to deviate from the zenith orientation by specifying three consecutive rotation angles. If no specific attitude adjustment is needed, these rotation angles should be set to 0 degrees.

Additionally, the code includes three detectors:
- GroundFieldOfViewDetector
- ElevationDetector
- BooleanDetector


In [3]:
import orekit
vm = orekit.initVM()

from orekit.pyhelpers import setup_orekit_curdir
setup_orekit_curdir()

from org.orekit.attitudes import NadirPointing, LofOffset
from org.orekit.bodies import OneAxisEllipsoid, GeodeticPoint
from org.orekit.frames import FramesFactory, LOFType, TopocentricFrame, Frame, Transform
from org.orekit.time import TimeScalesFactory, AbsoluteDate
from org.orekit.utils import Constants, IERSConventions, PVCoordinatesProvider
from org.orekit.propagation import SpacecraftState
from org.orekit.propagation.analytical.tle import TLE, TLEPropagator
from org.orekit.models.earth.tessellation import EllipsoidTessellator
from org.orekit.propagation.events import EventDetector, EventsLogger, ElevationDetector, GroundFieldOfViewDetector, BooleanDetector
from org.orekit.propagation.events.handlers import ContinueOnEvent, StopOnEvent
from org.orekit.geometry.fov import CircularFieldOfView, DoubleDihedraFieldOfView

from org.hipparchus.geometry.euclidean.threed import Vector3D, Line, RotationOrder, Rotation, RotationConvention
from org.hipparchus.geometry.spherical.twod import SphericalPolygonsSet

from math import radians, degrees, pi, sqrt, atan
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re

First call the frames and time scales to be used in the code and create Earth body.

In [5]:
UTC = TimeScalesFactory.getUTC()                               # Define UTC time scale.
ECI = FramesFactory.getEME2000()                               # Define ECI reference frame.
ECEF = FramesFactory.getITRF(IERSConventions.IERS_2010, True)  # Define ECEF reference frame.
TEME = FramesFactory.getTEME()                                 # Define TEME reference frame. 
ITRF = ECEF

R_earth  = Constants.WGS84_EARTH_EQUATORIAL_RADIUS             # Radius of earth
Mu_earth = Constants.WGS84_EARTH_MU                            # Gravitational parameter of earth
f_earth  = Constants.WGS84_EARTH_FLATTENING                    # Earth flattening value

earth = OneAxisEllipsoid(R_earth, f_earth, ITRF)               # Create earth here.

Here, we first take TLE as an input, and from the *TLEPropagator* method we recieve its PV to get Kepler Elements.

In [7]:
# ISS 25544
tle_line1 = "1 25544U 98067A   24265.15389185  .00020992  00000-0  38244-3 0  9995"
tle_line2 = "2 25544  51.6375 200.0394 0007352  15.0244 345.0962 15.49404240473433"
mytle = TLE(tle_line1, tle_line2)


initialDate = mytle.getDate()                    # This is the TLE epoch date read from first line.
finalDate = initialDate.shiftedBy(3600.0 * 72)   # Shift initial date by x hours.

print('Propagation initial date: ', initialDate)
print('Propagation final date: ', finalDate)

satellite_mass = 466615.0                        # Satellite mass [kg]
attitudeProvider = NadirPointing(TEME, earth)    # Also provide attitude of the satellite

SGP4 = TLEPropagator.selectExtrapolator(mytle, attitudeProvider, satellite_mass) 

Propagation initial date:  2024-09-21T03:41:36.25584Z
Propagation final date:  2024-09-24T03:41:36.25584Z


Geodetic coordinates (lat/long/alt) of the ground station must be defined below. Coordinate frame of the station is accepted as *TopocentricFrame*. This frame is associated to a position near the surface of a body shape.

The origin of the frame is at the defined geodetic point location, and the right-handed canonical trihedra is:

- X axis in the local horizontal plane (normal to zenith direction) and following the local parallel towards East
- Y axis in the horizontal plane (normal to zenith direction) and following the local meridian towards North
- Z axis towards Zenith direction

In [9]:
# Define the ground station location
latitude  = input('Latitude (degree):  ') 
latitude  = radians(float(latitude))
longitude = input('Longitude (degree):  ')
longitude = radians(float(longitude))
altitude  = input('Altitude (meter):  ')
altitude  = (float(altitude))

# Frame of the ground station is given as Topocentric Frame.
station_gp = GeodeticPoint(latitude, longitude, altitude)
station_frame = TopocentricFrame(earth, station_gp, "topo")

Latitude (degree):   35
Longitude (degree):   32
Altitude (meter):   0


In the following parts, we are going to create ground sensor with different FoV shapes using related lat/long/alt coordinates.

### Rotations with Respective Axes

The ground station frame is defined as topocentric. Based on this definition, the user can rotate this frame to achieve the desired attitude for the ground sensor.

In this code, **rotations are applied in the X-Y-Z order**. This means the topocentric frame is first rotated around the X axis, followed by the Y axis, and then the Z axis. While the rotation order can be modified, it's important to note that starting with a rotation around the Z axis is more prone to errors.

In [13]:
# Using MINUS for X and Y axes, and PLUS for Z axis in this code. Our notation becomes:
# (X = East, Y = North, Z = Zenith)
# which is the definition of topocentric frame in Orekit.
rotation1 = input('Enter the first rotation angle in degrees:  ')
rotation1 = radians(float(rotation1))
rotation2 = input('Enter the second rotation angle in degrees:  ')
rotation2 = radians(float(rotation2))
rotation3 = input('Enter the third rotation angle in degrees:  ')
rotation3 = radians(float(rotation3))

# If rotation vectors are entered PLUS instead of MINUS, then the correct XYZ notation becomes:
# (X = West, Y = South, Z = Zenith)

# First rotation
rotation = Transform(AbsoluteDate.ARBITRARY_EPOCH, Rotation(Vector3D.MINUS_I, rotation1, RotationConvention.VECTOR_OPERATOR))
myFoVFrame = Frame(station_frame, rotation, "myFovFrame")
# Second rotation
rotation = Transform(AbsoluteDate.ARBITRARY_EPOCH, Rotation(Vector3D.MINUS_J, rotation2, RotationConvention.VECTOR_OPERATOR))
myFoVFrame = Frame(myFoVFrame, rotation, "myFovFrame")
# Third rotation
rotation = Transform(AbsoluteDate.ARBITRARY_EPOCH, Rotation(Vector3D.PLUS_K, rotation3, RotationConvention.VECTOR_OPERATOR))
myFoVFrame = Frame(myFoVFrame, rotation, "myFovFrame")

Enter the first rotation angle in degrees:   0
Enter the second rotation angle in degrees:   0
Enter the third rotation angle in degrees:   0


After rotating our initial topocentric frame three times, we reach our desired attitude for the sensor attached to the ground station.

### Creating the Ground FoV Sensor 

This function provides flexibility in creating different types of sensor field-of-view (FoV) models, each tailored to specific sensor configurations and mission requirements:

- Circular FoV: Ideal for general coverage around the sensor.
- Rectangular FoV: Suitable for directional coverage.
- HPBW FoV: Typically used in antenna systems, taking into account frequency and sensor size.
 
This enhances the versatility of satellite tracking and detection simulations based on user-defined sensor configurations.

In [17]:
# Function to create Field-Of-View based on user input
def create_fov():
    print("Choose the type of Ground Field of View (FoV):")
    print("1- Circular FoV")
    print("2- Rectangular FoV")
    print("3- Half Power Beam Width (HPBW) FoV")

    fov_choice = input("Enter your choice (1/2/3): ")

    if fov_choice == "1":
        # Circular Field-Of-View
        circular_fov = input("Enter the half-angle of the circular FoV (degrees): ")
        circular_fov = float(circular_fov)
        fov_margin = 0.0
        imagerFOV = CircularFieldOfView(Vector3D.PLUS_K, radians(circular_fov), fov_margin)
        print("Circular FoV created.")

        
    elif fov_choice == "2":
        # Rectangular Field-Of-View
        verticalfov  = input('Enter the vertical half-angle (in degrees) of the cone formed by the target and the ground detector: ')
        verticalfov  = float(verticalfov)
        horizontalfov  = input('Enter the horizontal half-angle (in degrees) of the cone formed by the target and the ground detector: ')
        horizontalfov  = float(horizontalfov)
        fov_margin = 0.0

        # Create the Rectangular Field-Of-View
        imagerFOV = DoubleDihedraFieldOfView(Vector3D.PLUS_K,             # center
                                             Vector3D.PLUS_I,             # axis1
                                             radians(verticalfov),   # halfAperture1
                                             Vector3D.PLUS_J,             # axis2
                                             radians(horizontalfov), # halfAperture2
                                             fov_margin)                  # margin
        print("Rectangular FoV created.")
        
    elif fov_choice == "3":
        frequency = input('Enter the frequency of s/c in GHz:  ')
        frequency = float(frequency)
        diameter = input('Enter the diameter of the sensor in meters:  ')
        diameter = float(diameter)

        half_angle = ((70*0.3)/(frequency*diameter))/2
        xfov = half_angle
        fov_margin = 0.0

        imagerFOV = CircularFieldOfView(Vector3D.PLUS_K, radians(xfov), fov_margin)
        print("HPBW FoV created.")

    else:
        print("Invalid choice. Please select a valid option.")
        return create_fov()  # Recursion to prompt the user again if invalid input is given.

    return imagerFOV

# Call the function to create the appropriate Field-Of-View based on user input
imagerFOV = create_fov()

Choose the type of Ground Field of View (FoV):
1- Circular FoV
2- Rectangular FoV
3- Half Power Beam Width (HPBW) FoV


Enter your choice (1/2/3):  1
Enter the half-angle of the circular FoV (degrees):  25


Circular FoV created.


Defining FoV shape of the ground sensor and creating *GroundFieldOfViewDetector* together with *ElevationDetector* method. Finally these event detectors are combined with a *BooleanDetector*, and they are added to propagator for monitoring.

In [19]:
# Define Ground Field of View Detector
GFoVDetector = GroundFieldOfViewDetector(myFoVFrame, imagerFOV).withMaxCheck(5.0).withHandler(ContinueOnEvent())
GFoV_logger = EventsLogger()

# Define Elevation Detector
horizon_limit = radians(0.0)
eleDetector = ElevationDetector(station_frame).withConstantElevation(horizon_limit).withHandler(ContinueOnEvent())
ele_logger = EventsLogger()

#combinedDetector = BooleanDetector.andCombine([BooleanDetector.notCombine(GFoVDetector),eleDetector])
combinedDetector = BooleanDetector.andCombine([BooleanDetector.notCombine(GFoVDetector),eleDetector])
combined_logger = EventsLogger()

# Add the seperate detectors to the SGP4 propagator
SGP4.addEventDetector(GFoV_logger.monitorDetector(GFoVDetector))
SGP4.addEventDetector(ele_logger.monitorDetector(eleDetector))
SGP4.addEventDetector(combined_logger.monitorDetector(combinedDetector))

Now start propagating the TLE with SGP4. Here, propagation interval has to be sufficiently small because if it is large, detectors may miss certain events.

In [21]:
pos = []                                                       # position vector array to be filled.
SGP4 = PVCoordinatesProvider.cast_(SGP4)
###Start SGP4 propagation from initialDate up until finalDate
while (initialDate.compareTo(finalDate) <= 0.0):
    SGP4_pv = SGP4.getPVCoordinates(initialDate, ITRF)         # Get PV coordinates
    posSGP4 = SGP4_pv.getPosition()                            # But we only want position vector.
    pos.append((posSGP4.getX(),posSGP4.getY(),posSGP4.getZ())) # Get individual elements of position
    posSGP4 = pos
    initialDate = initialDate.shiftedBy(10.0)                  # Propagate with 10 sec intervals.

Finally the logged events are retrieved and printed in a readable way.

**Ground Field of View Detector** 

In [24]:
gfov_events = GFoV_logger.getLoggedEvents()            

entry_list = []
exit_list = []
duration_list = []

for event in gfov_events: 
    if not event.isIncreasing():
        gfov_entry = event.getState().getDate()
        entry_str = str(gfov_entry).replace("T", " ==> ")
        entry_list.append(entry_str)
    else:
        gfov_exit = event.getState().getDate()
        exit_str = str(gfov_exit).replace("T", " ==> ")
        exit_list.append(exit_str)

        gfov_duration = gfov_exit.durationFrom(gfov_entry) / 60  # Convert to minutes
        duration_list.append(gfov_duration)

# Create the DataFrame
df_gfov = pd.DataFrame({
    'GFoV Entry': entry_list,
    'GFoV Exit': exit_list,
    'Duration (min)': duration_list
})

# Display the DataFrame
display(df_gfov)


,GFoV Entry,GFoV Exit,Duration (min)
0,2024-09-21 ==> 13:10:20.94733983067657Z,2024-09-21 ==> 13:11:10.75206554302383Z,0.830079
1,2024-09-23 ==> 19:43:05.27890712249455Z,2024-09-23 ==> 19:43:39.32391538482981Z,0.567417


**Elevation Detector** 

In [26]:
ele_events = ele_logger.getLoggedEvents()            

entry_list = []
exit_list = []
duration_list = []

for event in ele_events: 
    if event.isIncreasing():
        ele_entry = event.getState().getDate()
        entry_str = str(ele_entry).replace("T", " ==> ")
        entry_list.append(entry_str)
    else:
        ele_exit = event.getState().getDate()
        exit_str = str(ele_exit).replace("T", " ==> ")
        exit_list.append(exit_str)

        ele_duration = ele_exit.durationFrom(ele_entry) / 60  # Convert to minutes
        duration_list.append(ele_duration)

# Create the DataFrame
df_ele = pd.DataFrame({
    'Elevation Entry': entry_list,
    'Elevation Exit': exit_list,
    'Duration (min)': duration_list
})

# Display the DataFrame
display(df_ele)

,Elevation Entry,Elevation Exit,Duration (min)
0,2024-09-21 ==> 11:31:04.3889233428157Z,2024-09-21 ==> 11:38:05.34339719106884Z,7.015908
1,2024-09-21 ==> 13:05:22.83878632906064Z,2024-09-21 ==> 13:16:11.78402997851241Z,10.815754
2,2024-09-21 ==> 14:43:15.87118737386536Z,2024-09-21 ==> 14:52:36.8991363854342Z,9.350466
3,2024-09-21 ==> 16:22:44.27060332866752Z,2024-09-21 ==> 16:29:20.85334813285234Z,6.609712
4,2024-09-21 ==> 18:00:41.84751944092243Z,2024-09-21 ==> 18:08:13.30536373354693Z,7.524297
5,2024-09-21 ==> 19:37:05.08287990613171Z,2024-09-21 ==> 19:47:24.35938859211115Z,10.321275
6,2024-09-21 ==> 21:13:47.77103591932551Z,2024-09-21 ==> 21:24:15.28081952796115Z,10.458496
7,2024-09-22 ==> 10:45:38.14996514107624Z,2024-09-22 ==> 10:48:21.06693410315203Z,2.715283
8,2024-09-22 ==> 12:17:42.36075050988782Z,2024-09-22 ==> 12:28:17.07420507222591Z,10.578558
9,2024-09-22 ==> 13:54:53.6714167933406Z,2024-09-22 ==> 14:04:52.82201177275074Z,9.985843


**Satellite Entry_Exit with Ground Sensor Field of View** 

In [28]:
comb_events = combined_logger.getLoggedEvents()            

entry_list = []
exit_list = []
duration_list = []

for event in comb_events: 
    if event.isIncreasing():
        comb_entry = event.getState().getDate()
        entry_str = str(comb_entry).replace("T", " ==> ")
        entry_list.append(entry_str)
    else:
        comb_exit = event.getState().getDate()
        exit_str = str(comb_exit).replace("T", " ==> ")
        exit_list.append(exit_str)

        comb_duration = comb_exit.durationFrom(comb_entry) / 60  # Convert to minutes
        duration_list.append(comb_duration)

# Create the DataFrame
df_comb = pd.DataFrame({
    'Combined Entry': entry_list,
    'Combined Exit': exit_list,
    'Duration (min)': duration_list
})

# Display the DataFrame
display(df_comb)

,Combined Entry,Combined Exit,Duration (min)
0,2024-09-21 ==> 13:10:20.94733983067657Z,2024-09-21 ==> 13:11:10.75206554302383Z,0.830079
1,2024-09-23 ==> 19:43:05.27890712249455Z,2024-09-23 ==> 19:43:39.32391538482981Z,0.567417
